In [1]:
!pip install optuna-integration[lightgbm]

import numpy as np
import pandas as pd
import lightgbm as lgb
import optuna
import optuna.integration.lightgbm as lgbm
from lightgbm import early_stopping,log_evaluation
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

train = pd.read_csv('/kaggle/input/playground-series-s4e12/train.csv',index_col = "id")
test = pd.read_csv('/kaggle/input/playground-series-s4e12/test.csv',index_col = "id")
sample_submission = pd.read_csv('/kaggle/input/playground-series-s4e12/sample_submission.csv',index_col = "id")

train.columns = [col.replace(" ", "_") for col in train.columns]
test.columns = [col.replace(" ", "_") for col in test.columns]

def date_preprocessing(df, yearT = 360):
    df = df.copy()
    df["Policy_Start_Date"] = pd.to_datetime(df["Policy_Start_Date"])
    days_in_month = df["Policy_Start_Date"].dt.days_in_month
    
    df["Year"] = df["Policy_Start_Date"].dt.year
    df["Month"] = df["Policy_Start_Date"].dt.month
    df["Day"] = df["Policy_Start_Date"].dt.day
    
    df["Sin_Month"] = np.sin(df["Month"] * np.pi * 2 / 12)
    df["Cos_Month"] = np.cos(df["Month"] * np.pi * 2 / 12)
    

    df["Day_of_Year"] = df["Policy_Start_Date"].dt.dayofyear
    df["Sin_Year_Day"] = np.sin(df["Day_of_Year"] * np.pi * 2 / yearT)
    df["Cos_Year_Day"] = np.cos(df["Day_of_Year"] * np.pi * 2 / yearT)
    
    df["Sin_Day"] = np.sin(df["Day"] * np.pi * 2 / days_in_month)
    df["Cos_Day"] = np.cos(df["Day"] * np.pi * 2 / days_in_month)

    df.drop("Policy_Start_Date",axis = 1,inplace = True)

    return df


train = date_preprocessing(train)
test = date_preprocessing(test)

cat_features = [col for col in train.columns if train[col].dtype == 'object']

for col in cat_features:
    train[col] = train[col].fillna("None").astype("category")
    test[col] = test[col].fillna("None").astype("category")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.4/97.4 kB 2.4 MB/s eta 0:00:00


In [2]:

def objective(trial):
    goss_params = {
                'random_state': 42,
                'objective': 'regression',
                'metric': 'rmse',
                'boosting_type': 'goss',
                'verbosity': -1,
                "verbose_eval": -1,
                'enable_categorical': True, 
                 "learning_rate": trial.suggest_float("learning_rate", 1e-4, 0.1, log=True),
                "num_leaves": trial.suggest_int("num_leaves", 20, 150),
                "max_depth": trial.suggest_int("max_depth", -1, 20), 
                "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 10, 100),
                "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
                "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
                "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
                "min_gain_to_split": trial.suggest_float("min_gain_to_split", 0.0, 1.0)}

    X,y = train.drop("Premium_Amount",axis = 1).copy(),train["Premium_Amount"].copy()
    kf = KFold(n_splits = 3,random_state = 42,shuffle = True)
    lgb_train_scores = []
    for fold, (train_idx, val_idx) in enumerate(kf.split(X, y)):
        y_log = np.log1p(y)
        X_train, X_valid = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_valid = y_log.iloc[train_idx], y_log.iloc[val_idx]
    
    
        lgb_train = lgb.Dataset(X_train, y_train, categorical_feature=cat_features)
        lgb_valid = lgb.Dataset(X_valid, y_valid, categorical_feature=cat_features, reference=lgb_train)

        model = lgb.train(
            goss_params,
            lgb_train,
            valid_sets=[lgb_train,lgb_valid],
            valid_names=["Training","Validation"],
            num_boost_round = 3000,
            callbacks=[lgb.early_stopping(stopping_rounds=50)])
         
        y_pred = model.predict(X_valid, num_iteration=model.best_iteration)
    
        y_pred = np.maximum(0,y_pred)
        y_valid = np.maximum(0,y_valid)
        
        rmsle = mean_squared_error(y_valid, y_pred,squared=False)
        lgb_train_scores.append(rmsle)

    return np.mean(lgb_train_scores)

study = optuna.create_study(sampler = optuna.samplers.TPESampler(),direction = "minimize")
study.optimize(objective,timeout = 3600 * 3)
print(study.best_params)

[I 2024-12-02 13:04:56,167] A new study created in memory with name: no-name-bd9f3c9e-3e3b-4eda-9fe9-414f573f5a22


Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[3000]	Training's rmse: 1.04883	Validation's rmse: 1.05343
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[3000]	Training's rmse: 1.04976	Validation's rmse: 1.05079
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[3000]	Training's rmse: 1.04974	Validation's rmse: 1.0519


[I 2024-12-02 13:24:17,408] Trial 0 finished with value: 1.0520406411045409 and parameters: {'learning_rate': 0.000975650712612266, 'num_leaves': 115, 'max_depth': 7, 'min_data_in_leaf': 84, 'feature_fraction': 0.6560268657630726, 'lambda_l1': 0.001055515495359659, 'lambda_l2': 0.1398323136718019, 'min_gain_to_split': 0.562399352061678}. Best is trial 0 with value: 1.0520406411045409.


Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[3000]	Training's rmse: 1.06529	Validation's rmse: 1.06721
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[3000]	Training's rmse: 1.06634	Validation's rmse: 1.06451
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[3000]	Training's rmse: 1.06606	Validation's rmse: 1.06612


[I 2024-12-02 13:31:09,648] Trial 1 finished with value: 1.0659500472107395 and parameters: {'learning_rate': 0.0013552196200012042, 'num_leaves': 45, 'max_depth': 2, 'min_data_in_leaf': 35, 'feature_fraction': 0.6318921741523428, 'lambda_l1': 2.717712769107199e-06, 'lambda_l2': 0.01127212730807297, 'min_gain_to_split': 0.2453836544899739}. Best is trial 0 with value: 1.0520406411045409.


...

Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[1405]	Training's rmse: 1.04063	Validation's rmse: 1.04787
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[1173]	Training's rmse: 1.04229	Validation's rmse: 1.04524
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[1257]	Training's rmse: 1.04208	Validation's rmse: 1.04589


[I 2024-12-02 15:53:55,680] Trial 16 finished with value: 1.0463342177153396 and parameters: {'learning_rate': 0.007848805418326386, 'num_leaves': 40, 'max_depth': 14, 'min_data_in_leaf': 26, 'feature_fraction': 0.7661082183511994, 'lambda_l1': 7.05505307355849e-05, 'lambda_l2': 0.003963981978794984, 'min_gain_to_split': 0.7448560908725992}. Best is trial 13 with value: 1.0459058765680267.


Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[3000]	Training's rmse: 1.04291	Validation's rmse: 1.04765
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[3000]	Training's rmse: 1.04377	Validation's rmse: 1.04496
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[3000]	Training's rmse: 1.04386	Validation's rmse: 1.0456


[I 2024-12-02 16:08:07,385] Trial 17 finished with value: 1.0460696356037935 and parameters: {'learning_rate': 0.002139725571524349, 'num_leaves': 37, 'max_depth': 9, 'min_data_in_leaf': 48, 'feature_fraction': 0.9048540411600601, 'lambda_l1': 1.6999343673904734, 'lambda_l2': 2.618744607760069e-05, 'min_gain_to_split': 0.8601458666753529}. Best is trial 13 with value: 1.0459058765680267.


{'learning_rate': 0.0074316004574371255, 'num_leaves': 48, 'max_depth': 13, 'min_data_in_leaf': 25, 'feature_fraction': 0.8859387413757838, 'lambda_l1': 7.427838968787608, 'lambda_l2': 2.9930367783188713e-05, 'min_gain_to_split': 0.7980368381910301}
